# Database Creator

Need to make/finish the functions to make the domain and subdomain lists

In [1]:
from security_domains import SECURITY_DOMAINS
from environments import ENVIRONMENTS

In [2]:
dev = ENVIRONMENTS.get('dev')
test = ENVIRONMENTS.get('test')
prod = ENVIRONMENTS.get('prod')

In [3]:
# WIP
def make_security_groups_from_domain_subdomain(domain, subdomain):
    pass


def add_domain(env=None, info=None, template=False):
    """Use template=True to output an example dictionary"""
    pass


def add_subdomain(env=None, domain=None, subdomains=None, template=False):
    """Use template=True to output an example dictionary"""
    sensitive = info.get('sensitive')
    for subdomain in subdomains:
        for sg in subdomain.get('security_groups'):
            add_security_group(
                env=env, domain=domain, subdomain=subdomain, sensitive=sensitive
            )
    pass

In [4]:
# ADD_SECURITY_GROUP IS WORKING
def add_security_group(env, domain, subdomain=None, sensitive=None):
    env_prefix = env.get('env_prefix')
    sensitive_suffix = env.get('sensitive_suffix')
    group_parts = [env_prefix, domain]
    if subdomain is not None:
        group_parts.append(subdomain)
    if sensitive is not None:
        if sensitive == 'replace':
            # add senstivie_suffix to end
            group_parts.append(sensitive_suffix)
        elif sensitive == 'add_extra':
            # add extra security group with sensitive_suffix
            group_parts_sen = group_parts.copy()
            group_parts_sen.append(sensitive_suffix)
            security_group_sen = '-'.join(group_parts_sen)
            add_group_sen_sql = f'''add group "{security_group_sen}";'''
        else:
            raise ValueError(
                f'sensitive must be either `add_extra` or `replace`, got `{sensitive}`'
            )
    security_group = '-'.join(group_parts)
    add_group_sql = f'''add group "{security_group}";'''
    if sensitive == 'add_extra':
        return [add_group_sql, add_group_sen_sql]
    else:
        return [add_group_sql]


In [5]:
add_security_group(
    env=dev, domain='widget', subdomain='master', sensitive='add_extra'
)


['add group "dev-widget-master";', 'add group "dev-widget-master-sensitive";']

In [6]:
add_security_group(
    env=ENVIRONMENTS.get('prod'), domain='api', subdomain='master', sensitive='replace'
)

['add group "prod-api-master-sensitive";']

In [7]:
# PRINT_DATABASE_DOMAINS IS WORKING
def print_database_domains(database, indent_chars=10):
    """prints the database domains, subdomains, and security groups"""

    def print_indent(item, level, type):
        indent_string = ' ' * level * indent_chars
        sep = '-'
        if type == 'security_domain':
            print_string = item.upper() + ' -- Security Domain'
            print(print_string)
            print(sep * len(print_string))
        elif type == 'domain':
            print_string = indent_string + item.upper() + ' -- Data Domain'
            print(print_string)
            print(
                ' ' * len(indent_string) + sep * (len(print_string) - len(indent_string))
            )
        elif type == 'subdomain':
            print_string = indent_string + item.upper() + ' -- Subdomain'
            print(print_string)
            print(
                ' ' * len(indent_string) + sep * (len(print_string) - len(indent_string))
            )
        elif type == 'security_group':
            print(indent_string + item)

    def display_and_interpret_domain(domains, level, type):
        for name, domain in domains.items():
            print()
            print_indent(name, level, type=type)
            interpret(domain, level)

    def interpret(thing, level=-1):
        security_domains = thing.get('security_domains')
        domains = thing.get('domains')
        subdomains = thing.get('subdomains')
        security_groups = thing.get('security_groups')
        if security_groups is not None:
            for security_group in security_groups:
                print_indent(f"'{security_group}'", level, type='security_group')
        if subdomains is not None:
            level += 1
            display_and_interpret_domain(
                domains=subdomains, level=level, type='subdomain'
            )
        if domains is not None:
            level += 1
            display_and_interpret_domain(
                domains=domains, level=level, type='domain'
            )
        if security_domains is not None:
            level += 1
            display_and_interpret_domain(
                domains=security_domains, level=level, type='security_domain'
            )

    return interpret(database)


In [8]:
print_database_domains(SECURITY_DOMAINS, indent_chars=8)


PRODUCT -- Security Domain
--------------------------

        WIDGET -- Data Domain
        ---------------------

                MASTER -- Subdomain
                -------------------
                'widget-master-data'
                'widget-master-data-sensitive'

                SALES -- Subdomain
                ------------------
                'widget-sales-actuals'
                'widget-sales-forecast'

                        NEW_WIDGET_SALES -- Subdomain
                        -----------------------------

                                AGGREGATED -- Subdomain
                                -----------------------

                                        ACTUALS -- Subdomain
                                        --------------------
                                        'widget-sales-new-aggregated-actuals'

                                USED_WIDGET_SALES -- Subdomain
                                ------------------------------

                          

In [9]:
# GET_DOMAIN_SECURITY_GROUPS IS WORKING
def get_domain_security_groups(database):
    """Get a list of data domain security groups"""
    security_group_list = []
    def reinterpret(domains):
        for name, domain in domains.items():
            interpret(domain)

    def interpret(thing):
        security_domains = thing.get('security_domains')
        domains = thing.get('domains')
        subdomains = thing.get('subdomains')
        security_groups = thing.get('security_groups')
        if security_groups is not None:
            for security_group in security_groups:
                if security_group != '':
                    security_group_list.append(security_group)
        if subdomains is not None:
            reinterpret(subdomains)
        if domains is not None:
            reinterpret(domains)
        if security_domains is not None:
            reinterpret(security_domains)
    
    interpret(database)
    return security_group_list


In [10]:
widgets_domain = SECURITY_DOMAINS['security_domains']['product']['domains']['widget']
for sg in get_domain_security_groups(widgets_domain):
    print(sg)

widget-master-data
widget-master-data-sensitive
widget-sales-actuals
widget-sales-forecast
widget-sales-new-aggregated-actuals
widget-pricing
widget-pricing-sensitive
widget-configuration
widget-configuration-sensitive
widget-inventory-volume
widget-inventory-volume-sensitive
widget-orders-actuals
widget-orders-actuals
quality-reports
manufacturing-defects
manufacturing-productivity
manufacturing-job-process
manufacturing-equipment
manufacturing-inspection
